In [58]:
import numpy as np
from gurobipy import *
import matplotlib.pyplot as plt
import pandas as pd
from itertools import combinations
import matplotlib.lines as lines

In [59]:
def Generate_Instance(num_cities):
    
    city_list = [np.random.uniform(0,50,2) for city in range(num_cities)]
    
    return city_list

In [60]:
def Plot_Instance(city_list):
    
    fig, axis = plt.subplots()
    df = pd.DataFrame({"x":[city_list[i][0] for i in range(len(city_list))],\
                      "y":[city_list[i][1] for i in range(len(city_list))]})
    
    df.plot(kind = "scatter", x= "x", y= "y", ax=axis)
    return None

def Plot_Instance_Solution(city_list, arcs):
    """
    Will plot your solution for debugging.  
    arcs is a list of lists where each inner list is the start and 
    end of trip taken.  So if x_{1,2}=1, then one element of arcs
    should be [1,2].
    """
    fig, axis = plt.subplots(figsize = (10,10))
    df = pd.DataFrame({"x":[city_list[i][0] for i in range(len(city_list))],\
                      "y":[city_list[i][1] for i in range(len(city_list))]})
    
    df.plot(kind = "scatter", x= "x", y= "y", ax=axis)
    
    for arc in arcs:
        start = arc[0]
        end =arc[1]
        
        axis.add_artist(lines.Line2D([city_list[start][0],city_list[end][0]],\
                                     [city_list[start][1],city_list[end][1]]))
        
    return None

In [61]:
def Get_Distance_Matrix(city_list):
    num_cities = len(city_list)
    distance_matrix = [[0 for j in range(num_cities)] for i in range(num_cities)]
    for i in range(num_cities):
        for j in range(num_cities):
            if i!=j:
                distance  = ((city_list[i][0] - city_list[j][0])**2 + (city_list[i][1] - city_list[j][1])**2)**0.5
                distance_matrix[i][j] = distance
    return distance_matrix
            

Code the constraint generation solution below.  You might find it useful to have a few helper functions. Just return the distance of the minimum distance tour. I have included the plotting functions just for fun and to help you debug.

In [62]:


def Solve_TSP_CG(num_cities, distance_matrix):
    
    subtour_list = []
    max_len=0
    while max_len<num_cities:
        objVal, arcs = TSP_1(num_cities,distance_matrix,subtour_list)
        subtour_adds = Find_Subtours(arcs,num_cities)
        subtour_list.extend(subtour_adds)
        max_len = (max([len(tour) for tour in subtour_list]))
    return objVal

def TSP_1(num_cities,distance_matrix,subtours):
    m = Model()
    x = {}
    for i in range(num_cities):
        for j in range(num_cities):
            x[i,j] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}")
            if i==j:
                m.addConstr(x[i,j] == 0)
    for i in range(num_cities):
        m.addConstr(quicksum(x[i,j] for j in range(num_cities)) == 1)
        m.addConstr(quicksum(x[j,i] for j in range(num_cities)) == 1) 
    m.setObjective(quicksum(x[i,j]*distance_matrix[i][j] for i in range(num_cities) for j in range(num_cities)),GRB.MINIMIZE)
    for subtour in subtours:
        if len(subtour) < num_cities:
            m.addConstr(quicksum(x[i, j] for i in subtour for j in subtour if i != j) <= len(subtour)-1)
    m.setParam("OutputFlag",False) 
    m.optimize()
    if m.status == GRB.OPTIMAL:
        arcs = [(i, j) for i in range(num_cities) for j in range(num_cities) if x[i, j].X > 0.5]
    min_dist = float(m.objVal)
    return min_dist, arcs

def Find_Subtours(arcs, num_cities):
    visited = [False] * num_cities
    subtours = []
    for start_city in range(num_cities):
        if not visited[start_city]:
            subtour = []
            current_city = start_city
            while not visited[current_city]:
                subtour.append(current_city)
                visited[current_city] = True
                for next_city in range(num_cities):
                    if (current_city, next_city) in arcs:
                        current_city = next_city
                        break
            if len(subtour) > 1:
                subtours.append(subtour)
    return subtours
    

In [63]:
np.random.seed(137)
num_cities =100
city_list = Generate_Instance(num_cities)
distance_matrix = Get_Distance_Matrix(city_list)
obj_1 =Solve_TSP_CG(num_cities, distance_matrix)
assert np.isclose(obj_1, 402.54)

In [64]:
#This one may take a minute or two to run
np.random.seed(17)
num_cities =150
city_list = Generate_Instance(num_cities)
distance_matrix = Get_Distance_Matrix(city_list)
obj_1 =Solve_TSP_CG(num_cities, distance_matrix)
assert np.isclose(obj_1, 457.36)